# Setup

### Import necessary modules and do some basic setup.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.utils import class_weight

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
import pickle
import pathlib
import hashlib
from pathlib import Path
#dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Dotenv
from dotenv import dotenv_values

In [ ]:
# Custom utils
from utils.utils_data import *
from utils.utils_plot import *
from utils.utils_RF import *
#from utils.utils_ml import *

### Define some paths and constants.

In [ ]:
config = dotenv_values(".env")

# Paths
PATH_ERA5 = config['PATH_ERA5']
PATH_EOBS = config['PATH_EOBS']

# Some constants
G = 9.80665

# Options
PRECIP_DATA = 'ERA5-low' # Options: ERA5-hi, ERA5-low, E-OBS
DATE_START = '1979-01-01'
DATE_END = '2021-12-31'
YY_TRAIN = [1979, 2015]
YY_TEST = [2016, 2021]
LEVELS = [300, 500, 700, 850, 925, 1000]
LONS_INPUT = [-25, 30]
LATS_INPUT = [30, 75]
LONS_PREC = [-25, 30]
LATS_PREC = [30, 75]
BATCH_SIZE = 64
PRECIP_XTRM = 0.95 # Percentile (threshold) for the extremes
CREATE_MASK_EOBS = False # only true when using E-OBS
RF_MAX_DEPTH = 3

In [ ]:
# Subsample domain for dev
#LONS_INPUT = [10, 15]
#LATS_INPUT = [40, 45]
#LONS_PREC = [10, 15]
#LATS_PREC = [40, 45]

# Data preparation

## Target variable: precipitation field

In [ ]:
# Load precipitation
if PRECIP_DATA == 'ERA5-hi':
    pr = get_nc_data(PATH_ERA5 + '/precipitation/orig_grid/daily/*nc', DATE_START, DATE_END, LONS_PREC, LATS_PREC)
    pr = pr.tp
elif PRECIP_DATA == 'ERA5-low':
    pr = get_nc_data(PATH_ERA5 + '/precipitation/day_grid1/*nc', DATE_START, DATE_END, LONS_PREC, LATS_PREC)
    pr = pr.tp
elif PRECIP_DATA in ['E-OBS', 'EOBS']:
    pr = get_nc_data(PATH_EOBS + '/eobs_1deg_v26.0e.nc', DATE_START, DATE_END, LONS_PREC, LATS_PREC)
    pr = pr.rr
    CREATE_MASK_EOBS = True
else:
    raise ValueError('Precipitation data not well defined')
pr['time'] = pd.DatetimeIndex(pr.time.dt.date)

In [ ]:
# Invert lat axis if needed
if pr.lat[0].values < pr.lat[1].values:
    pr = pr.reindex(lat=list(reversed(pr.lat)))

In [ ]:
# Create mask
mask = None
if CREATE_MASK_EOBS:
    if PRECIP_DATA in ['E-OBS', 'EOBS']:
        peobs = pr
    else:
        peobs = get_nc_data(PATH_EOBS + '/eobs_1deg_v26.0e.nc', DATE_START, DATE_END, LONS_PREC, LATS_PREC)
        peobs = peobs.rr
        if peobs.lat[0].values < peobs.lat[1].values:
            peobs = peobs.reindex(lat=list(reversed(peobs.lat)))
    mask = np.isnan(peobs[0,:,:])
    mask = np.invert(mask)
    mask.plot()
    mask = mask.to_numpy()

In [ ]:
# Compute the extreme exceedence
qq = xr.DataArray(pr).chunk(dict(time=-1)).quantile(PRECIP_XTRM, dim='time')
pr_xtrm = xr.DataArray(pr > qq)
pr_xtrm = pr_xtrm*1 # Transform to number

In [ ]:
qq.plot()

In [ ]:
# Extract coordinates for precip
lats_y = pr.lat.to_numpy()
lons_y = pr.lon.to_numpy()

## Input data: meteorological fields

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    # Load geopotential height
    z = get_era5_data(PATH_ERA5 + '/geopotential/day_grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
    z['time'] = pd.DatetimeIndex(z.time.dt.date)
    z = z.sel(level=LEVELS)

    # Get Z in geopotential height (m)
    z.z.values = z.z.values/G

    # Load temperature
    t = get_era5_data(PATH_ERA5 + '/temperature/day_grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
    t['time'] = pd.DatetimeIndex(t.time.dt.date)

    # Load relative humidity
    rh = get_era5_data(PATH_ERA5 + '/relative_humidity/day_grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
    rh['time'] = pd.DatetimeIndex(rh.time.dt.date)
    rh = rh.sel(level=LEVELS)

    # Load total column water
    tcw = get_era5_data(PATH_ERA5 + '/total_column_water/day_grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
    tcw['time'] = pd.DatetimeIndex(tcw.time.dt.date)

    # Load wind components
    u = get_era5_data(PATH_ERA5 + '/U_wind/day_grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
    u['time'] = pd.DatetimeIndex(u.time.dt.date)
    v = get_era5_data(PATH_ERA5 + '/V_wind/day_grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
    v['time'] = pd.DatetimeIndex(v.time.dt.date)

# Checking dimensions
print('dimension of pr:', pr.dims)
print('dimension of z', z.dims)
print('dimension of t:', t.dims)
print('dimension of rh:', rh.dims)
print('dimension of tcw:', tcw.dims)
print('dimension of u:', u.dims)
print('dimension of v:', v.dims)


In [ ]:
# Merge arrays
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    X = xr.merge([z, t, rh, tcw, u, v])
X

In [ ]:
# Invert lat axis if needed
if X.lat[0].values < X.lat[1].values:
    X = X.reindex(lat=list(reversed(X.lat)))
    
# Get axes
lats_x = X.lat
lons_x = X.lon

### Split data and transform

In [ ]:
# Split into training and test
X_train_full = X.sel(time=slice(f'{YY_TRAIN[0]}-01-01', f'{YY_TRAIN[1]}-12-31'))
X_test = X.sel(time=slice(f'{YY_TEST[0]}-01-01', f'{YY_TEST[1]}-12-31'))

pr_train_full = pr.sel(time=slice(f'{YY_TRAIN[0]}-01-01', f'{YY_TRAIN[1]}-12-31'))
pr_test = pr.sel(time=slice(f'{YY_TEST[0]}-01-01', f'{YY_TEST[1]}-12-31'))

pr_xtrm_train_full = pr_xtrm.sel(time=slice(f'{YY_TRAIN[0]}-01-01', f'{YY_TRAIN[1]}-12-31'))
pr_xtrm_test = pr_xtrm.sel(time=slice(f'{YY_TEST[0]}-01-01', f'{YY_TEST[1]}-12-31'))

In [ ]:
# Create a data generator
dic = {'z': LEVELS,
   't': LEVELS,
   'r': LEVELS,
   'tcwv': None,
   'u': LEVELS,
   'v': LEVELS}

from utils.utils_ml import *

YY_VALID = 2005
# we might not need to split into valid
dg_train = DataGeneratorWithExtremes(X_train_full.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                     pr_train_full.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                     pr_xtrm_train_full.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                     dic, batch_size=BATCH_SIZE, load=True)
dg_valid = DataGeneratorWithExtremes(X_train_full.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                     pr_train_full.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                     pr_xtrm_train_full.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                     dic, mean=dg_train.mean, std=dg_train.std,
                                     batch_size=BATCH_SIZE, load=True)
dg_test = DataGeneratorWithExtremes(X_test, pr_test, pr_xtrm_test, dic,
                                    mean=dg_train.mean, std=dg_train.std,
                                    batch_size=BATCH_SIZE, load=True, shuffle=False)


In [ ]:
i_shape = dg_train.X.shape
o_shape = dg_train.y.shape

print(f'X shape: {i_shape}')
print(f'y shape: {o_shape}')

In [ ]:
chunk = {'lon':5, 'lat':5}

# Train a random forest classifier

In [ ]:
X_train_dask = dg_train.X.chunk(chunk)
y_train_dask = dg_train.y_xtrm.chunk(chunk)

In [ ]:
# models file
clf_file = f'tmp/trained_classifiers_RF_{PRECIP_DATA}_{PRECIP_XTRM}_{RF_MAX_DEPTH}.pkl'

In [ ]:
if os.path.isfile(clf_file):
    print('open models')
    #open models
    with open(clf_file, 'rb') as f:
        crfs = pickle.load(f)
else:
    
#%%time
    crfs = xr.apply_ufunc(
        train_rf_classifier_model,
        X_train_dask, y_train_dask, RF_MAX_DEPTH,
        vectorize=True,
        dask = 'parallelized',
        input_core_dims=[['time', 'level'], ['time'], []],  # reduce along these dimensions
        output_dtypes=[object]
    ).compute()
    # save the models
    with open(clf_file, 'wb') as output:
            pickle.dump(crfs, output, pickle.HIGHEST_PROTOCOL)

### Predict for the testing and training datasets

In [ ]:
%%time
y_test_preds = xr.apply_ufunc(
    apply_rf_classifier_model, 
    crfs, dg_test.X,
    vectorize=True,
    input_core_dims=[[],['time', 'level']],
    output_dtypes=['object'] # change the output type
).compute()

In [ ]:
%%time
y_test_preds_proba = xr.apply_ufunc(
    apply_rf_classifier_model_proba, 
    crfs, dg_test.X,
    vectorize=True,
    input_core_dims=[[],['time', 'level']],
    output_dtypes=['object'] # change the output type
).compute()

In [ ]:
%%time
y_train_preds = xr.apply_ufunc(
    apply_rf_classifier_model, 
    crfs, dg_train.X,
    vectorize=True,
    input_core_dims=[[],['time', 'level']],
    output_dtypes=['object'] # change the output type
).compute()

In [ ]:
# Create Xarray with the predictions

In [ ]:
y_pred_xtrm_test = create_xarray_frompred(y_test_preds, dg_test.y.shape[0], lats_y, lons_x)
yprob_pred_xtrm_test = create_xarray_frompred(y_test_preds_proba, dg_test.y.shape[0], lats_y, lons_x)
y_pred_xtrm_train = create_xarray_frompred(y_train_preds, dg_train.y.shape[0], lats_y, lons_x)

In [ ]:
# as an example
y_pred_xtrm_test[800,:,:].plot()

In [ ]:
yprob_pred_xtrm_test[800,:,:].plot()

#### Calculate scores (these are also calculated later in the notebook)

In [ ]:
%%time
auc = xr.apply_ufunc(
    eval_rf_auc, 
    crfs, dg_test.X,dg_test.y_xtrm,
    vectorize=True,
    input_core_dims=[[],['time', 'level'],['time']],
    output_core_dims=[[]],
    output_dtypes=[float] # change the output type
).compute()

In [ ]:
%%time
precision = xr.apply_ufunc(
    eval_rf_precision, 
    crfs, dg_test.X,dg_test.y_xtrm,
    vectorize=True,
    input_core_dims=[[],['time', 'level'],['time']],
    output_core_dims=[[]],
    output_dtypes=[float] # change the output type
).compute()


In [ ]:
%%time
recall = xr.apply_ufunc(
    eval_rf_recall, 
    crfs, dg_test.X,dg_test.y_xtrm,
    vectorize=True,
    input_core_dims=[[],['time', 'level'],['time']],
    output_core_dims=[[]],
    output_dtypes=[float] # change the output type
).compute()

In [ ]:
# Apply mask before plotting if data is E-OBS

In [ ]:
def filter_mask(mask, metric):
    """filter with the mask"""
    nan_mask=mask*1.0
    nan_mask[mask==0]=np.nan
    
    return(metric*nan_mask)

In [ ]:
if CREATE_MASK_EOBS:
    precision = filter_mask(mask, precision)
    recall = filter_mask(mask, recall)
    

In [ ]:
# take a look to the results

In [ ]:
fig, axes = plt.subplots(figsize=(24, 3.5), ncols=3, nrows=1)
plot_map(axes[0], lons_y, lats_y, auc, title="AUC", vmin=0, vmax=1)
plot_map(axes[1], lons_y, lats_y, precision, title="precision", vmin=0, vmax=1)
plot_map(axes[2], lons_y, lats_y, recall, title="recall", vmin=0, vmax=1)

In [ ]:
# Metrics-Spatially averages 
metrics_prec_xtrm = pd.DataFrame([(auc.mean().values, recall.mean().values, precision.mean().values)], columns=['auc', 'recall', 'precision'])

In [ ]:
print('RF-averages scores:')
print(metrics_prec_xtrm)

In [ ]:
# Reshape predictions to create maps
#preds_reshaped = np.zeros(dg_test.y.shape)

# Train a random forest regressor

In [ ]:
# adapt similar functions to the classification case

In [ ]:
X_train_dask = dg_train.X.chunk(chunk)
y_train_dask = dg_train.y.chunk(chunk)

In [ ]:
# models file
rfs_file = f'tmp/trained_regressors_RF_{PRECIP_DATA}_{PRECIP_XTRM}_{RF_MAX_DEPTH}.pkl'

In [ ]:
def train_rf_regress_model(X, y, max_depth=4):
    if np.any(np.isnan(y)):
        
        return None
        
    try:
        rgf =  RandomForestRegressor(max_depth=max_depth, random_state=42).fit(X, y)
        #clf.fit(X, y)
        print('|', end='')
        return rgf
    except:
        #print('Failed to create the model')
        return None

In [ ]:
if os.path.isfile(rfs_file):
    #print('open models')
    #open models
    with open(rfs_file, 'rb') as f:
        rfs = pickle.load(f)
else:
    
    rfs = xr.apply_ufunc(
        train_rf_regress_model,
        X_train_dask, y_train_dask, RF_MAX_DEPTH,
        vectorize=True,
        dask = 'parallelized',
        input_core_dims=[['time', 'level'], ['time'], []],  # reduce along these dimensions
        output_dtypes=[object]
    ).compute()
    # save the models
    with open(rfs_file, 'wb') as output:
            pickle.dump(rfs, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
# calculate predictions
y_test_preds_reg = xr.apply_ufunc(
    apply_rf_regress_model, 
    rfs, dg_test.X,
    vectorize=True,
    input_core_dims=[[],['time', 'level']],
    output_core_dims=[[]],
    output_dtypes=[object] # change the output type
).compute()

In [ ]:
# calculate predictions
y_train_preds_reg = xr.apply_ufunc(
    apply_rf_regress_model, 
    rfs, dg_train.X,
    vectorize=True,
    input_core_dims=[[],['time', 'level']],
    output_core_dims=[[]],
    output_dtypes=[object] # change the output type
).compute()

In [ ]:
# Create matrix of predictions
preds_test_reg_matrix = create_xarray_frompred(y_test_preds_reg, dg_test.y.shape[0], lats_y, lons_x)
preds_train_reg_matrix = create_xarray_frompred(y_train_preds_reg, dg_train.y.shape[0], lats_y, lons_x)

In [ ]:
preds_test_reg_matrix[800,:,:].plot()

#### calculate rmse

In [ ]:
#evaluate--check --not working
rmse = xr.apply_ufunc(
    eval_rf_mse, 
    rfs, dg_test.X,dg_test.y,
    vectorize=True,
    input_core_dims=[[],['time', 'level'],['time']],
    #output_core_dims=[[]],
   # output_dtypes=[object] # change the output type
).compute()

In [ ]:
fig, axes = plt.subplots(figsize=(12, 6), ncols=1, nrows=1)
plot_map(axes, lons_y, lats_y, rmse, title="RMSE")


In [ ]:
# Spatial average of MSE
# Metrics-Spatially averages 
rmse.mean().values

# Predict and assess on the test period

In [ ]:
# Predict the amount of precipitation
df_prec = pd.DataFrame(columns = ['id', 'name',
                                  'train_pr_rmse', 'test_pr_rmse', 
                                  'train_xtrm_precision', 'test_xtrm_precision', 
                                  'train_xtrm_recall', 'test_xtrm_recall'],index=[1])
df_prec.at[df_prec.index[0], 'id'] = 'RF'
df_prec.at[df_prec.index[0], 'name'] = 'Regressor'
df_xtrm = pd.DataFrame(columns = ['id', 'name', 
                                  'train_xtrm_roc_auc', 'test_xtrm_roc_auc', 
                                  'train_xtrm_precision', 'test_xtrm_precision', 
                                  'train_xtrm_recall', 'test_xtrm_recall'],index=[1])
df_xtrm.at[df_xtrm.index[0], 'id'] = 'RF'
df_xtrm.at[df_xtrm.index[0], 'name'] = 'Classifier'

In [ ]:
### Result for precipitation amounts, training and testing

In [ ]:
apply_for_prec = True

if apply_for_prec:
    print('Train results')
    precision_train, recall_train, rmse_train = analyze_predictions(preds_train_reg_matrix, dg_train, qq, lons_y, lats_y, pred_xtrm=False, show_plots=False)
    df_prec.at[df_prec.index[0], 'train_xtrm_precision'] = precision_train
    df_prec.at[df_prec.index[0], 'train_xtrm_recall'] = recall_train
    df_prec.at[df_prec.index[0], 'train_pr_rmse'] = rmse_train
    print('Test results')
    precision, recall, rmse = analyze_predictions(preds_test_reg_matrix, dg_test, qq, lons_y, lats_y, pred_xtrm=False, show_plots=False)
    df_prec.at[df_prec.index[0], 'test_xtrm_precision'] = precision
    df_prec.at[df_prec.index[0], 'test_xtrm_recall'] = recall
    df_prec.at[df_prec.index[0], 'test_pr_rmse'] = rmse
 

In [ ]:
df_prec

In [ ]:
### Result for precipitation extremes, training and testing

In [ ]:
apply_for_xtrm = True

if apply_for_xtrm:
    print('Train results')
    precision_train, recall_train, auc_train = analyze_predictions(y_pred_xtrm_train, dg_train, qq, lons_y, lats_y, pred_xtrm=True, show_plots=False)
    df_xtrm.at[df_xtrm.index[-1], 'train_xtrm_precision'] = precision_train
    df_xtrm.at[df_xtrm.index[-1], 'train_xtrm_recall'] = recall_train
    df_xtrm.at[df_xtrm.index[-1], 'train_xtrm_roc_auc'] = auc_train

    print('Test results')
    precision, recall, auc = analyze_predictions(y_pred_xtrm_test, dg_test, qq, lons_y, lats_y, pred_xtrm=True, show_plots=False)
    df_xtrm.at[df_xtrm.index[-1], 'test_xtrm_precision'] = precision
    df_xtrm.at[df_xtrm.index[-1], 'test_xtrm_recall'] = recall
    df_xtrm.at[df_xtrm.index[-1], 'test_xtrm_roc_auc'] = auc
    

In [ ]:
df_xtrm

In [ ]:
# plot fig.1 and 2 but only using truth and RF
n_rows = 2
m_id = "RF"
fig, axs = plt.subplots(n_rows, 3, figsize=(10, n_rows*3))
# For the model take predictions
y_pred = preds_test_reg_matrix
y_pred_bool = y_pred > qq.to_numpy().squeeze()

# Extract true values
y_xtrm = dg_test.y_xtrm.to_numpy().squeeze()
y_prec = dg_test.y.to_numpy().squeeze()

# Get the index of the max # of extremes
i_max_obs = np.argmax(np.sum(y_xtrm, axis=(1,2)))

vmax = np.max(y_prec[i_max_obs])

plot_map(axs[0, 0], lons_y, lats_y, y_prec[i_max_obs], title="Prec. value - truth", vmin=0, vmax=vmax, show_colorbar=False, cmap=mpl.cm.YlGnBu)
plot_map(axs[0, 1], lons_y, lats_y, y_xtrm[i_max_obs], title="Prec. extreme - truth", vmin=0, vmax=1, show_colorbar=False)
plot_map(axs[0, 2], lons_y, lats_y, y_xtrm[i_max_obs], title="Prec. extreme - truth", vmin=0, vmax=1, show_colorbar=False)


# Plot the model
plot_map(axs[1, 0], lons_y, lats_y, y_pred[i_max_obs], title=f"Prec. value - {m_id}", vmin=0, vmax=vmax, show_colorbar=False, cmap=mpl.cm.YlGnBu)
plot_map(axs[1, 1], lons_y, lats_y, y_pred_bool[i_max_obs], title=f"Prec. extreme - {m_id}", vmin=0, vmax=1, show_colorbar=False)
plot_map(axs[1, 2], lons_y, lats_y, yprob_pred_xtrm_test[i_max_obs], title=f"Prec. extreme - {m_id}", vmin=0, vmax=1, show_colorbar=False)
# Save the plot
plt.tight_layout()
plt.savefig('figures/plot_model_RF95th_comparison.pdf')